# 🚀 Text Generation Apps using Python (Completions Only)

This notebook showcases **14 text generation use cases** using OpenAI **Completions API**.  
Each section includes a description, the Python code, and expected output.

---

## 📑 Table of Contents
1. [Summarization](#-Use-Case-1-Summarization)  
2. [Sentiment Classification](#-Use-Case-2-Sentiment-Classification)  
3. [Multilingual Generation + Translation](#-Use-Case-3-Multilingual-Generation--Translation)  
4. [Semantic Interpretation of Idiom](#-Use-Case-4-Semantic-Interpretation-of-Idiom)  
5. [Factual Recall / Explanatory Response](#-Use-Case-5-Factual-Recall--Explanatory-Response)  
6. [Code Generation & Explanation](#-Use-Case-6-Code-Generation--Explanation)  
7. [Conversational Agent (FAQ Assistant)](#-Use-Case-7-Conversational-Agent-FAQ-Assistant)  
8. [Style Transfer / Tone Adaptation](#-Use-Case-8-Style-Transfer--Tone-Adaptation)  
9. [Data-to-Text Generation](#-Use-Case-9-Data-to-Text-Generation)  
10. [Creative Writing](#-Use-Case-10-Creative-Writing)  
11. [Question Generation](#-Use-Case-11-Question-Generation)  
12. [Entity Extraction with Explanation](#-Use-Case-12-Entity-Extraction-with-Explanation)  
13. [Paraphrasing / Rewriting](#-Use-Case-13-Paraphrasing--Rewriting)  
14. [Email / Document Drafting](#-Use-Case-14-Email--Document-Drafting)  

---


## Setup

- Ensure your `.env` file contains `OPENAI_API_KEY` (never hardcode secrets).
- Use `python-dotenv` to load environment variables securely.
- Import and initialize the `OpenAI` client with your API key.
- Define a reusable helper function `get_completion(prompt, ...)` to call the OpenAI Completions API with configurable model, temperature, and max tokens.
- Prefer environment variables for model selection and temperature overrides.
- Example setup and usage are shown in the next code cell.

In [ ]:
# Install required packages (skip if already installed)
# %pip install openai tiktoken python-dotenv

import os
import time
import random
import logging
from dataclasses import dataclass
from typing import Optional, Sequence

from dotenv import load_dotenv
from openai import OpenAI, APIError, RateLimitError

# -------------------------------------------------------------------
# Logging (avoid reconfiguring if handlers already exist)
# -------------------------------------------------------------------
if not logging.getLogger().handlers:
    logging.basicConfig(level=logging.INFO,
                        format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# -------------------------------------------------------------------
# Environment loading & helpers
# -------------------------------------------------------------------
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in environment or .env file")

DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo-instruct")


def _env_float(name: str, default: float) -> float:
    raw = os.getenv(name)
    if not raw:
        return default
    # Ignore Windows TEMP style paths accidentally picked up
    if os.path.sep in raw:
        return default
    try:
        val = float(raw)
        if 0 <= val <= 1:
            return val
    except ValueError:
        pass
    return default


def _env_int(name: str, default: int, min_val: int = 1, max_val: int = 4000) -> int:
    raw = os.getenv(name)
    if not raw:
        return default
    try:
        val = int(raw)
        if min_val <= val <= max_val:
            return val
    except ValueError:
        pass
    return default


def _get_client() -> OpenAI:
    # Could be extended for per-request customization later
    return OpenAI(api_key=OPENAI_API_KEY)


client = _get_client()

# -------------------------------------------------------------------
# Result container
# -------------------------------------------------------------------


@dataclass
class CompletionResult:
    text: str
    model: str
    prompt_tokens: Optional[int] = None
    completion_tokens: Optional[int] = None
    total_tokens: Optional[int] = None
    retries: int = 0

# -------------------------------------------------------------------
# Core helper with light retries
# -------------------------------------------------------------------


def get_completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    max_tokens: Optional[int] = None,
    temperature: Optional[float] = None,
    stop: Optional[Sequence[str]] = None,
    log_response: bool = False,
    max_retries: int = 3,
    backoff_base: float = 1.5,
) -> Optional[CompletionResult]:
    """Call OpenAI Completions API with resilience & env overrides.

    Environment overrides:
      OPENAI_MODEL          -> default model if not passed
      OAI_TEMP / OAI_TEMPERATURE -> temperature (0..1)
      OAI_MAX_TOKENS        -> max_tokens upper bound
    """
    if not prompt or not prompt.strip():
        raise ValueError("Prompt must be a non-empty string")

    # Apply env fallbacks only if caller did not specify
    if temperature is None:
        temperature = _env_float(
            "OAI_TEMP", _env_float("OAI_TEMPERATURE", 0.7))
    if max_tokens is None:
        max_tokens = _env_int("OAI_MAX_TOKENS", 200, 1, 4096)

    attempt = 0
    last_error: Optional[Exception] = None
    while attempt <= max_retries:
        try:
            start = time.time()
            resp = client.completions.create(
                model=model,
                prompt=prompt.strip(),
                max_tokens=max_tokens,
                temperature=temperature,
                stop=list(stop) if stop else None,
            )
            latency = (time.time() - start) * 1000
            choice = resp.choices[0]
            usage = getattr(resp, "usage", None)
            if log_response:
                logger.debug("Raw response: %s", resp)
            logger.info(
                "completion model=%s tokens=%s latency=%.1fms retries=%d",  # noqa: E501
                model,
                getattr(usage, "total_tokens", "?"),
                latency,
                attempt,
            )
            return CompletionResult(
                text=choice.text.strip(),
                model=model,
                prompt_tokens=getattr(usage, "prompt_tokens", None),
                completion_tokens=getattr(usage, "completion_tokens", None),
                total_tokens=getattr(usage, "total_tokens", None),
                retries=attempt,
            )
        except (RateLimitError, APIError) as e:
            last_error = e
            if attempt == max_retries:
                logger.error("Max retries reached (%d): %s", max_retries, e)
                break
            sleep_for = backoff_base ** attempt + random.uniform(0, 0.25)
            logger.warning("Transient error (%s). Retrying in %.2fs (attempt %d/%d)",
                           e.__class__.__name__, sleep_for, attempt + 1, max_retries)
            time.sleep(sleep_for)
            attempt += 1
            continue
        except Exception as e:  # Non-retryable
            logger.error("Completion failed: %s", e)
            last_error = e
            break
    return None


# -------------------------------------------------------------------
# Example usage (kept minimal for notebook demonstration)
# -------------------------------------------------------------------
if __name__ == "__main__":
    test_prompt = "Write a haiku about AI and the future."
    result = get_completion(test_prompt, log_response=True)
    if result:
        print("✅ Completion result:\n", result.text)
        if result.total_tokens is not None:
            print(
                f"(tokens: {result.total_tokens}, retries: {result.retries})")
    else:
        print("❌ No response received.")

## ✅ Use Case 1: Summarization
**Input:**  
Sparse vector feature indexing allows scalable search over events by turning categorical attributes into an extremely wide, mostly empty vector...

**Sample Output:**  
"It converts categorical features into sparse vectors for scalable search. The trade-off is efficiency vs. memory/compute cost at high cardinality."

In [ ]:
# ✅ Use Case 1: Summarization Code

technical_paragraph = """
Sparse vector feature indexing allows scalable search over events by turning 
categorical attributes into an extremely wide, mostly empty vector. Each distinct 
token (e.g. event_type:click, browser:mobile) maps to a coordinate set to 1 while 
all others remain 0. This removes unintended ordering between categories and keeps 
distance metrics meaningful, but at very high cardinality memory and compute 
overhead grow rapidly, motivating hashing tricks or learned embeddings.
""".strip()

summary_prompt = f"Summarize the following paragraph in 2 crisp sentences focusing on purpose and trade-offs:\n\n{technical_paragraph}\n\nSummary:"

# Call completions API using helper
summary = get_completion(
    summary_prompt, max_tokens=120, temperature=0.4)

print("✅ Summary:", summary)

## ✅ Use Case 2: Sentiment Classification
**Input:**  
"The user interface feels sluggish, but the reporting features are fantastic."

**Sample Output:**  
"Neutral"


In [ ]:
# ✅ Use Case 2: Sentiment Classification (Multiple Inputs)

# Sample inputs
input_texts = [
    "The user interface feels sluggish, but the reporting features are fantastic.",  # Neutral
    "I love the new dashboard design, it’s clean and very intuitive.",               # Positive
    "The system keeps crashing frequently, making it hard to use."                   # Negative
]

# Loop through each input and classify sentiment
for input_text in input_texts:
    sentiment_prompt = f"""
    Classify the sentiment of the following text strictly as Positive, Negative, or Neutral.
    If the text contains both positive and negative aspects, classify it as Neutral.

    Text: "{input_text}"

    Sentiment:
    """

    # Call completions API using helper (returns CompletionResult or None)
    result = get_completion(
        sentiment_prompt,
        max_tokens=20,
        temperature=0  # deterministic output
    )

    if result and result.text:
        sentiment_text = result.text.strip()
    else:
        sentiment_text = "(no response)"

    print(f"✅ Text: {input_text}")
    print(f"✅ Sentiment: {sentiment_text}\n")

## ✅ Use Case 3: Multilingual Generation + Translation
**Input:**  
"Artificial Intelligence will transform education."

**Sample Output:**  
- French: "L'intelligence artificielle transformera l'éducation."  
- Spanish: "La inteligencia artificial transformará la educación."  
- Hindi: "कृत्रिम बुद्धिमत्ता शिक्षा को बदल देगी।"


In [ ]:
# ✅ Use Case 3: Multilingual Generation + Translation

input_text = "Artificial Intelligence will transform education."
languages = ["French", "Spanish", "Hindi"]

for lang in languages:
    translation_prompt = f"Translate the following text into {lang}:\n\n{input_text}"
    translation = get_completion(
        translation_prompt, max_tokens=60, temperature=0.3)
    print(f"✅ {lang}: {translation}\n\n")

## ✅ Use Case 4: Semantic Interpretation of Idiom
**Input:**  
"Kick the bucket"

**Sample Output:**  
"It means someone has died."


In [ ]:
# ✅ Use Case 4: Semantic Interpretation of Idiom

idioms = [
    "kick the bucket",
    "spill the beans",
    "hit the sack",
    "once in a blue moon",
    "break the ice",
]

for idiom in idioms:
    prompt = f"""
Explain the meaning of the idiom below in one concise, literal-friendly plain English sentence.
Avoid extra commentary or cultural notes unless essential.
Return ONLY the meaning (no leading labels).

Idiom: "{idiom}"

Meaning:
""".strip()

    meaning = get_completion(
        prompt,
        model="gpt-3.5-turbo-instruct",
        max_tokens=60,
        temperature=0  # deterministic for stable definitions
    )

    if meaning:
        print(f"🗣️ {idiom} -> {meaning}\n\n")
    else:
        print(f"❌ No response for idiom: {idiom}\n\n")

## ✅ Use Case 5: Factual Recall / Explanatory Response
**Input:**  
"Why is the sky blue?"

**Sample Output:**  
"The sky looks blue because sunlight is scattered by air molecules. Blue light is scattered more strongly than other colors, so we mostly see blue."

In [ ]:
# ✅ Use Case 5: Factual Recall / Explanatory Response

question = "Why is the sky blue?"

prompt = f"""
Provide a concise scientific explanation (2 sentences) for the question below.
Avoid fluff; focus on the underlying physical phenomenon.

Question: {question}

Answer:
""".strip()

result = get_completion(prompt, max_tokens=120, temperature=0.2)

if result and result.text:
    print("❓ Question:", question)
    print("🧪 Answer:", result.text.strip())
    if result.total_tokens is not None:
        print(f"(tokens: {result.total_tokens}, retries: {result.retries})")
else:
    print("❌ No response received.")

## ✅ Use Case 6: Code Generation & Explanation
**Input:**  
"Write a Python function that checks if a number is prime."

**Sample Output:**  
```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
````

**Explanation:**
"The function checks divisibility from 2 up to √n. If any divisor is found, it returns False; otherwise, True."


In [ ]:
# ✅ Use Case 6: Code Generation & Explanation

problem = "Write a Python function that checks if a number is prime."

prompt = f"""
You are a senior Python engineer.
Generate clean, production-quality code that fulfills the requirement below.
After the code, add a single concise explanation sentence.
Use exactly this delimiter line before the explanation:
---EXPLANATION---

Requirement:
{problem}

Return ONLY:
1. A single Python code block
2. The delimiter line
3. One concise explanation sentence
""".strip()

result = get_completion(prompt, max_tokens=220, temperature=0)

if not result or not result.text:
    print("❌ No response received.")
else:
    raw = result.text.strip()
    # Attempt to split code and explanation
    parts = raw.split("---EXPLANATION---", 1)
    code_part = raw
    explanation = None
    if len(parts) == 2:
        code_part, explanation = parts[0].strip(), parts[1].strip()

    print("🧩 Generated Code:")
    print(code_part)
    if explanation:
        print("\n📝 Explanation:", explanation)
    if result.total_tokens is not None:
        print(f"\n(tokens: {result.total_tokens}, retries: {result.retries})")

## ✅ Use Case 7: Conversational Agent (FAQ Assistant)

**Input:**
"What is the purpose of Git?"

**Sample Output:**
"Git is a version control system that tracks changes in code, helps collaboration, and manages project history."

In [ ]:
# ✅ Use Case 7: Conversational Agent (FAQ Assistant)

# A tiny in-memory FAQ knowledge base (could be loaded from file/db in real apps)
faq_kb = {
    "What is Git?": "Git is a distributed version control system for tracking changes in source code.",
    "Why use version control?": "It enables history, collaboration, branching, and safe experimentation.",
    "What is a commit?": "A commit records a snapshot of your staged changes with a message.",
    "What is branching?": "Branching lets you diverge from the main line of development to work independently.",
    "What is a pull request?": "A pull request proposes merging changes from one branch into another, supporting review.",
}

# Simulated user questions (some exact, some paraphrased / partially overlapping)
user_queries = [
    "What is the purpose of Git?",
    "Explain branching in simple terms.",
    "Why should teams adopt version control?",
]

system_instruction = (
    "You are a concise technical FAQ assistant. Answer ONLY from the provided FAQ entries. "
    "If the answer is not present, reply with 'I don't have that information.' Keep answers to one or two sentences."
)

history: list[tuple[str, str]] = []  # (user_question, answer)

for q in user_queries:
    # Build a retrieval-aware prompt by listing KB and prior turns.
    kb_block = "\n".join(f"Q: {k}\nA: {v}" for k, v in faq_kb.items())
    convo_block = "\n".join(
        # last few turns
        f"User: {hq}\nAssistant: {ha}" for hq, ha in history[-4:])

    prompt = f"""
{system_instruction}

FAQ Knowledge Base:
{kb_block}

Conversation So Far:
{convo_block if convo_block else '(no previous turns)'}

User Question: {q}

Answer (draw ONLY from KB or say you don't have it):
""".strip()

    result = get_completion(prompt, max_tokens=110, temperature=0.2)

    if result and result.text:
        answer = result.text.strip()
    else:
        answer = "(no response)"

    history.append((q, answer))
    print(f"🙋 User: {q}")
    print(f"🤖 Assistant: {answer}\n")

# Show a compact transcript summary
print("--- Transcript Summary ---")
for uq, ans in history:
    print(f"Q: {uq}\nA: {ans}\n")


## ✅ Use Case 8: Style Transfer / Tone Adaptation

**Input:**
"Please confirm receipt of this document at the earliest."

**Sample Output:**
"Hey, just let me know when you get this doc!"

In [ ]:
## ✅ Use Case 8: Style Transfer / Tone Adaptation

## ✅ Use Case 9: Data-to-Text Generation

**Input:**
Region: North
Sales: \$50,000
Increase: 10%

**Sample Output:**
"The North region generated \$50,000 in sales, reflecting a 10% increase compared to the previous period."


## ✅ Use Case 10: Creative Writing

**Input:**
"A robot learning to paint"

**Sample Output:**
"The robot dipped its brush in bright colors, each stroke a clumsy attempt at beauty. Slowly, it learned to create sunsets and flowers. Its creators realized the machine had found its soul in art."

## ✅ Use Case 11: Question Generation

**Input:**
"The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south."

**Sample Output:**

1. What is the largest ocean on Earth?
2. How deep is the Pacific Ocean compared to other oceans?
3. Which oceans border the Pacific to the north and south?


## ✅ Use Case 12: Entity Extraction with Explanation

**Input:**
"Apple announced a new partnership with Tesla to develop AI-powered batteries in California."

**Sample Output:**

* **Apple** – Technology company forming a partnership.
* **Tesla** – Automotive/energy company collaborating on batteries.
* **California** – Location where the initiative is happening.

## ✅ Use Case 13: Paraphrasing / Rewriting

**Input:**
"The proliferation of interconnected devices has exponentially increased the complexity of ensuring cybersecurity across distributed networks."

**Sample Output:**
"With more connected devices, keeping networks secure has become much harder."



## ✅ Use Case 14: Email / Document Drafting

**Input:**
"Announce a team meeting for Friday at 3 PM to discuss the quarterly results."

**Sample Output:**
"Subject: Team Meeting – Friday at 3 PM
Hi Team,
We will have a meeting on Friday at 3 PM to review our quarterly results. Please make sure to attend.
Best regards,
\[Your Name]"

